In [45]:
import pandas as pd
import os
import string 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datetime import datetime
import numpy as np
import multiprocessing





#import tensorflow
#import pickle
#import os
from joblib import Parallel, delayed
import multiprocessing
import torch


In [2]:
q = """ SELECT * FROM `wagon-bootcamp-328013.trip.comment` LIMIT 1000"""
data = pd.read_gbq(q,project_id="wagon-bootcamp-328013")

/Users/nicolasmanoharan/.pyenv/versions/3.8.12/envs/FOBO/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


In [4]:
def cleaner(data):
    data['review_clean'] = data['commentaire'].apply(clear_data_text)
    #data['date'] = data['date'].apply(change_date)
    #data['rate'] = data['rate'].apply(keep_digit)
    data['review_sentences'] = data['review_clean'].apply(splitter)
    df_exploded = data.explode('review_sentences').reset_index(drop=True)
    return df_exploded

def splitter(text):
    tmp = re.split('[.?!]', text)
    tmp = filter(None, tmp)
    return list(tmp)


def clear_data_text(text, language='french',deep_clean=False):
    """
    Returns a clean text as a string. Default language = english.
    Cleaning includes : removing punct, digit and \\n, stopwords, informations
    related to author review, and lowercase.
    """
    #remove before the begin of the comment
    for i in range(5):
        pos = 0
        pos = text.find('\n')
        text = text[pos + 1:]

    text=text.replace('\n',' ')
    text=text.replace('/','')

    if deep_clean:
        #remove punctuation
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')

    if True:
        # remove number
        text = ''.join(word for word in text if not word.isdigit())

    #pass in lowercase
    text = text.lower()

    if deep_clean:
        #remove stop words
        stop_words = set(stopwords.words(language))

        #text to list of word
        word_tokens = word_tokenize(text)

        #list to string
        text = " ".join([w for w in word_tokens if not w in stop_words])

    #remove after the end of the comment
    text=text.strip()
    text=text.replace('  ',' ')
    if text.endswith('useful funny cool'):
        text = text[:-len('useful funny cool')]
        text = text.strip()
    if text.startswith('updated review'):
        text = text.split('useful funny cool', 1)[0]
        text = text[len('updated review'):]
        text = text.strip()
    if text.startswith('photos'):
        text = text[len('photos'):]
        text = text.strip()
    if text.startswith('photo'):
        text = text[len('photo'):]
        text = text.strip()
    text = text.strip()
    return text



In [5]:
data['review_clean'] = data['commentaire'].apply(clear_data_text)
data['review_sentences'] = data['review_clean'].apply(splitter)
df_exploded = data.explode('review_sentences').reset_index(drop=True)

In [30]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

In [52]:
len(df_exploded)

4359

In [31]:
%%time
test = [i for i in df_exploded["review_sentences"] if len(i) < 500]



embeddings = model.encode(test)

CPU times: user 7min 49s, sys: 7.53 s, total: 7min 57s
Wall time: 7min 41s


In [57]:
df_exploded

,date,note,titre,commentaire,adresse,nom,review_clean,review_sentences
0,2021-07-27 00:00:00+00:00,2.0,Cuisine médiocre à prix trop élevé,Nos plats étaient de qualité médiocre et nous ...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,nos plats étaient de qualité médiocre et nous ...,nos plats étaient de qualité médiocre et nous ...
1,2021-07-27 00:00:00+00:00,2.0,Cuisine médiocre à prix trop élevé,Nos plats étaient de qualité médiocre et nous ...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,nos plats étaient de qualité médiocre et nous ...,on trouve facilement meilleur pour moins cher...
2,2021-07-27 00:00:00+00:00,2.0,Cuisine médiocre à prix trop élevé,Nos plats étaient de qualité médiocre et nous ...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,nos plats étaient de qualité médiocre et nous ...,je e recommande pas
3,2016-02-17 00:00:00+00:00,2.0,complètement surfait,le type de restau qui est une pure tromperie s...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,le type de restau qui est une pure tromperie s...,le type de restau qui est une pure tromperie s...
4,2016-02-17 00:00:00+00:00,2.0,complètement surfait,le type de restau qui est une pure tromperie s...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,le type de restau qui est une pure tromperie s...,"toute l'apparence de la bonne adresse, mais c..."
...,...,...,...,...,...,...,...,...
4354,2016-09-17 00:00:00+00:00,5.0,"Mini pizza, maxi goût",Passage un midi le samedi. Très bon accueil ma...,"11 Rue Danielle Casanova Ouver, 75001 Paris Fr...",73. Mipi,passage un midi le samedi. très bon accueil ma...,passage un midi le samedi
4355,2016-09-17 00:00:00+00:00,5.0,"Mini pizza, maxi goût",Passage un midi le samedi. Très bon accueil ma...,"11 Rue Danielle Casanova Ouver, 75001 Paris Fr...",73. Mipi,passage un midi le samedi. très bon accueil ma...,très bon accueil malgré notre arrivée tardive
4356,2016-09-17 00:00:00+00:00,5.0,"Mini pizza, maxi goût",Passage un midi le samedi. Très bon accueil ma...,"11 Rue Danielle Casanova Ouver, 75001 Paris Fr...",73. Mipi,passage un midi le samedi. très bon accueil ma...,les pizzas plus petites (cm) permettent d'en ...
4357,2016-09-17 00:00:00+00:00,5.0,"Mini pizza, maxi goût",Passage un midi le samedi. Très bon accueil ma...,"11 Rue Danielle Casanova Ouver, 75001 Paris Fr...",73. Mipi,passage un midi le samedi. très bon accueil ma...,la pâte ainsi que les garnitures sont excelle...


In [62]:
df_full = df_exploded

In [64]:
df_all_resto = df_full.groupby('nom').agg({ 'note':'mean',
                                                'commentaire':'nunique'
                                            })


In [65]:



embeddings = model.encode(df_exploded["review_sentences"] )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 

In [72]:
def compute_sim_df(text, embedding, n_prox=3000, min_review=0):
    input_encoded = model.encode(text)
    embedding_split = np.array_split(embedding,
                                     multiprocessing.cpu_count() - 1)

    similarities_parra = Parallel(n_jobs=-1)(
        delayed(util.cos_sim)(input_encoded, split_embedding)
        for split_embedding in embedding_split)
    # // powa !
    similarities = torch.cat(similarities_parra, 1)
    #similarities = util.cos_sim(input_encoded, np.array(embedding))

    df_sim = df_full.assign(sim=similarities.T)

    if n_prox:
        df_sentences = df_sim.sort_values('sim', ascending=False)[:n_prox]
    else:
        df_sentences = df_sim.sort_values('sim', ascending=False)

    df_agg = df_sentences.groupby('nom').agg({
        'note': 'mean',
        'commentaire': 'nunique',
        #'review_sentences':'first',
        #'review_clean':lambda txt: ' // '.join(txt),
        'sim': 'mean'
    })

    df_resto = df_agg.merge(df_all_resto,
                            on='nom',
                            how='left',
                            suffixes=('_filtered', '_all')).reset_index()

    df_resto['ratio'] = df_resto['commentaire_filtered'] / df_resto['commentaire_all']

    df_resto = df_resto.sort_values('ratio')
    df_resto = df_resto[df_resto['commentaire_all'] > min_review]

    df_final = df_sentences.merge(df_resto,
                                  on='nom',
                                  how='left',
                                  suffixes=('_s', '_r'))

    df_final['metric'] = df_final['ratio'] * df_final['sim_r'] * df_final[
        'note_filtered'] / 5

    df_final.drop(columns=['commentaire'], inplace=True)

    return df_final


In [79]:
compute_sim_df("accueil",embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,date,note,titre,adresse,nom,review_clean,review_sentences,sim_s,note_filtered,commentaire_filtered,sim_r,note_all,commentaire_all,ratio,metric
0,2017-12-31 00:00:00+00:00,5.0,le 16 décembre 2017,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,accueil bon! nous avons attendu volontiers apr...,accueil bon,0.918642,4.561922,320,0.219398,4.498994,326,0.981595,0.196491
1,2019-10-28 00:00:00+00:00,5.0,Très bonne adresse,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,bon accueil. ambiance chaleureuse. les plats s...,bon accueil,0.912197,4.561922,320,0.219398,4.498994,326,0.981595,0.196491
2,2016-02-20 00:00:00+00:00,5.0,Restaurant Tradi,"4 Rue Du Mail, 75002 Paris France",7. TRADI,"ce restaurant est parfait, bonne ambiance on y...",bonne accueil,0.901370,4.856736,337,0.227757,4.768895,344,0.979651,0.216730
3,2017-03-16 00:00:00+00:00,5.0,Toujours top comm d hab ',"4 Rue Du Mail, 75002 Paris France",7. TRADI,super accueil ! le cuisinier aime travailler à...,super accueil,0.843027,4.856736,337,0.227757,4.768895,344,0.979651,0.216730
4,2018-11-26 00:00:00+00:00,5.0,Excellente nourriture,"10 Rue de Port-Mahon, 75002 Paris France",14. YO,restaurant excellent. service et saveurs au re...,très bon accueil,0.841203,4.561922,320,0.219398,4.498994,326,0.981595,0.196491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2016-11-05 00:00:00+00:00,4.0,Sacré lobster roll,"9 rue du Nil, 75002 Paris France",109. FTG,le lobster roll est une tuerie ! la sauce est ...,les frites en accompagnement sont excellentes...,0.046621,3.709534,158,0.174612,3.615124,172,0.918605,0.119002
2996,2014-01-06 00:00:00+00:00,2.0,accueil et amabilité à revoir,"9 rue du Nil, 75002 Paris France",109. FTG,surtout n'y aller pas le dimanche matin ! vous...,surtout n'y aller pas le dimanche matin,0.046535,3.709534,158,0.174612,3.615124,172,0.918605,0.119002
2997,2016-04-25 00:00:00+00:00,5.0,"Super la nourriture thaï, un bon service","10 Rue de Port-Mahon, 75002 Paris France",14. YO,nous avons vraiment beaucoup aimé cet hôtel et...,le curry crevettes était très savoureux,0.046428,4.561922,320,0.219398,4.498994,326,0.981595,0.196491
2998,2020-10-23 00:00:00+00:00,5.0,Excellente soirée !,"4 Rue Du Mail, 75002 Paris France",7. TRADI,restaurant idéal pour une soirée en couple ! r...,"repas excellent, personnel accueillant et cha...",0.046375,4.856736,337,0.227757,4.768895,344,0.979651,0.216730
